In [1]:
#Bruker CNN arkitektur kombinert med PyTorch 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [4]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [5]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [6]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [7]:
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [9]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [10]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [11]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [12]:
class ConvModel(nn.Module):
    def __init__(self):

        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)

        self.fc1 = nn.Linear(11*11*64, 128)
        self.fc2 = nn.Linear(128,2)


    def forward(self, x:Tensor):
        x = self.conv1(x)
        x = F.relu(x) #to activate function above

        x = F.max_pool2d(x,2)

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x,2)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
 
        return x

In [13]:
def trainFunction(config=None):

    #init wandb
    with wandb.init(project="PyTorch", name="convmodel", config=config):
        config = wandb.config

        #initialize variables
        convModel = ConvModel().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(convModel.parameters(), lr=config.learning_rate)
        #initialize variables end


        #data
        trainLoader = DataLoader(train, shuffle=True, batch_size=100)
        testLoader = DataLoader(test, shuffle=True, batch_size=100)
        #data end


        wandb.watch(convModel, criterion, log='all')

        
        #START
        for epoch in range(50):

            #training variables
            trainRunningLoss = 0.0
            correct = 0
            y_pred = []
            y_true = []
            #training loop
            convModel.train()
            for i, data in enumerate(trainLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                output = convModel(inputs.permute(0,3,1,2))
                output1 = (torch.max(torch.exp(output.to(device)), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = criterion(output, labels.type(torch.LongTensor).to(device))
                loss.backward()
                optimizer.step()
                
                trainRunningLoss += loss.item()
            #training loop end
            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            trainAccuracy = correct / len(y_true)
            #training variables end

            #test variables
            testRunningLoss = 0.0
            y_pred = []
            y_true = []
            #test loop
            convModel.eval()
            for j, data in enumerate(testLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                output = convModel(inputs.permute(0,3,1,2))# Feed Network

                output1 = (torch.max(torch.exp(output.to(device)), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = criterion(output, labels.type(torch.LongTensor).to(device))
                testRunningLoss += loss.item()
            #test loop end

            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            testAccuracy = correct / len(y_true)
            epochLoss = trainRunningLoss/len(bhArray)
            #test variables end

            #wandb log
            wandb.log({"Train epoch_loss":epochLoss, "Test epoch_loss": testRunningLoss, "Train accuracy": trainAccuracy,"Test accuracy": testAccuracy})

        #END


In [14]:
# #Weights and biases set up
# wandb.init(
#     # set the wandb project where this run will be logged
#     name="convmodel",
#     project="PyTorch",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "epochs": 50,
#     'batch_size': 100,
#     }
# )


In [15]:


sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        # 'batch_size': {'values': [50, 60, 70, 80, 90, 100]},
        'learning_rate': {'values': [0.01, 0.02, 0.05, 0.1]}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="test")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 8kof07td
Sweep URL: https://wandb.ai/g13hvl2023/test/sweeps/8kof07td


In [16]:
wandb.agent(sweep_id, function=trainFunction, count=5)

wandb: Agent Starting Run: jkakoqyj with config:
wandb: 	learning_rate: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 591291 (g13hvl2023). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Test accuracy,▁▅▆▄██▇▇▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Test epoch_loss,▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇███████
Train accuracy,▁▄▅▅▆▆▆▆▇▇▇▇████████████████████████████
Train epoch_loss,█▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test accuracy,0.806
Test epoch_loss,106.953
Train accuracy,0.99991
Train epoch_loss,1e-05


wandb: Agent Starting Run: rtg6dhqg with config:
wandb: 	learning_rate: 0.02
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Test accuracy,▁▃▃▆████▆▇▇▆▆▃▆▆▅▅▆▆▆▇▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
Test epoch_loss,▂▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▃▃▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇████
Train accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇███▇▇███████████████████
Train epoch_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test accuracy,0.80133
Test epoch_loss,132.62922
Train accuracy,0.99987
Train epoch_loss,1e-05


wandb: Agent Starting Run: 0iv6qera with config:
wandb: 	learning_rate: 0.05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Test accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train accuracy,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test accuracy,0.50547
Test epoch_loss,nan
Train accuracy,0.49818
Train epoch_loss,nan


wandb: Agent Starting Run: dh9kxrs6 with config:
wandb: 	learning_rate: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Test accuracy,▁██▁██▁██▁▁███▁█▁█▁▁▁▁█▁▁██▁██▁▁▁███▁██▁
Test epoch_loss,▁▁▁▂▁▁▂▁▁▅▅▁▁▁▄▁▂▁█▃▂▅▁▂▅▁▁▃▁▁▄▂▃▁▁▁▂▁▁▅
Train accuracy,▃▄▅▄▄▇▅▅▅▅▄▆▅▁▂▇▂▅▆▆▂█▄▂▆▂▄▂▇▅▃▂▄▂▇▅▅▆▃▄
Train epoch_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test accuracy,0.49453
Test epoch_loss,52.04307
Train accuracy,0.4988
Train epoch_loss,0.0104


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [17]:
# trainFunction()